In [1]:
import warnings
warnings.filterwarnings('ignore')

import os
import sys
import numpy as np
import pandas as pd

BACKTESTING_MODULE_PATH = os.path.abspath('../../backtest')
sys.path.insert(1, BACKTESTING_MODULE_PATH)
from backtesting.backtesting import Backtest, Strategy

METHOD_MODULE_PATH = os.path.abspath('../..')
sys.path.insert(1, METHOD_MODULE_PATH)
import method.Ticker as Ticker
import method.SaleRules as _sr
import method.algofuncs as _af
path = os.getcwd()

Loading BokehJS ...

In [2]:
path = os.getcwd()
class FollowTheTrend(Strategy):
    def init(self):
        self.buy_price = 0
        self._index = 0
    def next(self):
        prices = self.data.Close
        opens = self.data.Open
        volumes = self.data.Volume
        if len(self.data.Volume) > 5:
            last_price = prices[-1]
            if self.buy_price == 0 and Ticker.isFollowTrendingV2(prices, volumes, opens[-1], 2.5):
                self.buy_price = last_price
                self.buy()
            if self.buy_price != 0 and (_sr.takeProfitByPercent(5, last_price, self.buy_price) or _sr.shouldCutLossByPercent(8, last_price, self.buy_price)):
                self.position.close()
                self.buy_price = 0

In [21]:
DATA_PATH = os.path.abspath('../../vn-stock-data/VNX/')
vn30_ticker = _af.getListVN30()
reports = [['Ticker', 'Avg. Trade [%]', 'Equity Final [$]', 'Return [%]', 'Sharpe Ratio', 'Win Rate [%]', 'Trades']]
for ticker_id in vn30_ticker:
    ticker = _af.get_pricing_by_path(DATA_PATH + '/' + ticker_id+'.csv', '2018-01-01', '2020-10-05')
    bt = Backtest(ticker, FollowTheTrend, commission=.005, exclusive_orders=False)
    stats = bt.run()
    if stats['_trades'].shape[0]:
        reports = np.append(reports, [[ticker_id, stats['Avg. Trade [%]'], stats['Equity Final [$]'], stats['Return [%]'], stats['Sharpe Ratio'], stats['Win Rate [%]'], stats['_trades'].shape[0]]], axis=0)
ls = pd.DataFrame(data=reports[1:,1:],index=reports[1:,0],columns=reports[0,1:])
ls_s = ls.sort_values(by=['Avg. Trade [%]'], ascending=False, na_position='first')
ls_s


,Avg. Trade [%],Equity Final [$],Return [%],Sharpe Ratio,Win Rate [%],Trades
PNJ,4.740242756319235,12027.501389000005,20.275013890000047,0.7830764684889643,100.0,4
GAS,4.6071378856979095,11441.554310000001,14.415543100000013,0.7951389665612043,100.0,3
REE,1.872258990889697,10970.050091000003,9.700500910000027,0.3097866349303066,80.0,5
MWG,1.6622384560823544,11239.073979500008,12.390739795000082,0.30555137943706845,71.42857142857143,7
VCB,1.479847665409384,10762.256048500005,7.622560485000049,0.2233232009206966,80.0,5
VNM,0.8491504363261848,10255.419030500003,2.5541903050000294,0.15849179155721124,66.66666666666666,3
VJC,0.5174959770094745,10158.900064500003,1.5890006450000327,0.07012648081375635,66.66666666666666,3
GMD,0.5084675520624415,10414.55220400001,4.145522040000105,0.09832130432723447,75.0,8
CTD,-9.636929350316192,8171.596257999998,-18.284037420000022,0.0,0.0,2
SBT,-5.666800470787758,8396.294333000005,-16.037056669999945,0.0,33.33333333333333,3


60